In [ ]:
https://github.com/Tencent/HunyuanVideo

#screen
apt-get update && apt-get install -y screen
screen

#开启学术加速
source /etc/network_turbo
#关闭学术加速
unset http_proxy && unset https_proxy

In [ ]:
#安装虚拟环境到数据盘
#执行以下命令设置将虚拟环境安装到/root/autodl-tmp/conda/envs， 包缓存到/root/autodl-tmp/conda/pkgs
mkdir -p /root/autodl-tmp/conda/pkgs
conda config --add pkgs_dirs /root/autodl-tmp/conda/pkgs
mkdir -p /root/autodl-tmp/conda/envs
conda config --add envs_dirs /root/autodl-tmp/conda/envs
# 创建新的site_packages目录
mkdir -p /path/to/new/site_packages
# 设置环境变量PYTHONUSERBASE
export PYTHONUSERBASE=/path/to/new
# 安装包到新的site_packages目录
pip install --user package_name

In [ ]:
# 1. Prepare conda environment
conda env create -f environment.yml

# 2. Activate the environment
conda activate HunyuanVideo

# 3. Install pip dependencies
python -m pip install -r requirements.txt

# 4. Install flash attention v2 for acceleration (requires CUDA 11.8 or above)
python -m pip install ninja
python -m pip install git+https://github.com/Dao-AILab/flash-attention.git@v2.5.9.post1

In [ ]:
#安装 flash attention v2 用于加速 (要求 CUDA 11.8 或更高)
python -m pip install git+https://github.com/Dao-AILab/flash-attention.git@v2.5.9.post1

#所有模型默认存储在HunyuanVideo/ckpts中
#要下载浑源视频模型，请先安装huggingface-插件
python -m pip install "huggingface_hub[cli]"
#huggingface-cli download tencent/HunyuanVideo --local-dir ./ckpts
huggingface-cli download tencent/HunyuanVideo --local-dir /root/autodl-tmp/ckpts

#推荐社区用户使用Xtuer提供的llava-llama-3-8b，可通过以下命令下载
#MLLM模型（text_encoder文件夹）
cd HunyuanVideo/ckpts
huggingface-cli download xtuner/llava-llama-3-8b-v1_1-transformers --local-dir ./llava-llama-3-8b-v1_1-transformers
huggingface-cli download xtuner/llava-llama-3-8b-v1_1-transformers --local-dir /root/autodl-tmp/ckpts/llava-llama-3-8b-v1_1-transformers


#为了节省模型加载的GPU内存资源，我们将llava-llama-3-8b-v1_1-transformers的语言模型部分分离到text_encoder中。
cd HunyuanVideo
python hyvideo/utils/preprocess_text_encoder_tokenizer_utils.py --input_dir /root/autodl-tmp/ckpts/llava-llama-3-8b-v1_1-transformers --output_dir /root/autodl-tmp/ckpts/text_encoder

#CLIP模型（text_encoder_2文件夹）
cd HunyuanVideo/ckpts
huggingface-cli download openai/clip-vit-large-patch14 --local-dir /root/autodl-tmp/ckpts/text_encoder_2

In [ ]:
cd HunyuanVideo

python sample_video.py \
    --video-size 544 960 \
    --video-length 129 \
    --infer-steps 10 \
    --prompt "一个老人坐在海边长椅上看日落，背影显得孤寂悲凉" \
    --flow-reverse \
    --use-cpu-offload \
    --save-path ./results

In [ ]:
参数  默认值 描述
--prompt    没有一 用于生成视频的 prompt
--video-size    720 1280    生成视频的高度和宽度
--video-length  129 生成视频的帧数
--infer-steps   50  生成时采样的步数
--embedded-cfg-scale    6.0 文本的控制强度
--flow-shift    7.0 推理时 timestep 的 shift 系数，值越大，高噪区域采样步数越多
--flow-reverse  假   如果相反，则从t=1 -> t=0学习/采样
--neg-prompt    没有一 负向词
--seed  0   随机种子
--use-cpu-offload   假   启用 CPU offload，可以节省显存
--save-path ./结果    保存路径

In [ ]:
#Run a Gradio Server
python gradio_server.py --flow-reverse
# set SERVER_NAME and SERVER_PORT manually
# SERVER_NAME=0.0.0.0 SERVER_PORT=8081 python3 gradio_server.py --flow-reverse

ssh -CNg -L 6006:127.0.0.1:6006 root@123.125.240.150 -p 42151
ssh -CNg -L 8081:127.0.0.1:8081 root@connect.bjc1.seetacloud.com -p 10514 
#ssh -p 10514 root@connect.bjc1.seetacloud.com